<a href="https://colab.research.google.com/github/yenee/0320/blob/main/energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [2]:
def direct_supply(year):
    url = 'https://www.trec.org.tw/certification_trade_situation/direct_supply'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'Origin': 'https://www.trec.org.tw',
        'Referer': url,
    }
    data_url = 'https://www.trec.org.tw/certification_trade_situation/direct_supply/data'

    with requests.Session() as s:
        response = s.get(url=url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        token_item = soup.find('input', {'name': "_token"})
        if token_item is not None:
            token = token_item['value']
            print('token:', token)
        else:
            print("Token not found!")
        form_data = {
            "draw": 1,
            "columns[0][data]": "DT_RowIndex",
            "columns[0][name]": "DT_RowIndex",
            "columns[0][searchable]": "false",
            "columns[0][orderable]": "false",
            "columns[0][search][value]": "",
            "columns[0][search][regex]": "false",
            "columns[1][data]": "seller_name",
            "columns[1][name]": "seller_name",
            "columns[1][searchable]": "false",
            "columns[1][orderable]": "true",
            "columns[1][search][value]": "",
            "columns[1][search][regex]": "false",
            "columns[2][data]": "buyer_name",
            "columns[2][name]": "buyer_name",
            "columns[2][searchable]": "false",
            "columns[2][orderable]": "true",
            "columns[2][search][value]": "",
            "columns[2][search][regex]": "false",
            "columns[3][data]": "energy",
            "columns[3][name]": "energy",
            "columns[3][searchable]": "false",
            "columns[3][orderable]": "true",
            "columns[3][search][value]": "",
            "columns[3][search][regex]": "false",
            "columns[4][data]": "type",
            "columns[4][name]": "type",
            "columns[4][searchable]": "false",
            "columns[4][orderable]": "false",
            "columns[4][search][value]": "",
            "columns[4][search][regex]": "false",
            "columns[5][data]": "amount",
            "columns[5][name]": "amount",
            "columns[5][searchable]": "false",
            "columns[5][orderable]": "true",
            "columns[5][search][value]": "",
            "columns[5][search][regex]": "false",
            "columns[6][data]": "detail",
            "columns[6][name]": "detail",
            "columns[6][searchable]": "false",
            "columns[6][orderable]": "false",
            "columns[6][search][value]": "",
            "columns[6][search][regex]": "false",
            "order[0][column]": "1",
            "order[0][dir]": "asc",
            "start": 0,
            "length": -1,
            "search[value]": "",
            "search[regex]": "false",
            "year": year,
            "_token": token
        }

        response = s.post(url=data_url, data=form_data, headers=headers)
        print('status_code:', response.status_code)

        data = json.loads(response.text)
        print(data['recordsTotal'])
        df = pd.DataFrame(data['data'])

        def clean_seller_name(seller_name):
            return seller_name.split('<br/>')[0]

        df['seller_name'] = df['seller_name'].apply(clean_seller_name)
        df['year'] = year
        return df


In [3]:
year = [2024, 2023, 2022, 2021, 2020]
dfs = []

for y in year:
    df = direct_supply(y)
    time.sleep(5)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
selected_columns = ['year', 'seller_name', 'case_name', 'buyer_name', 'energy', 'type', 'amount']
selected_df = combined_df[selected_columns]

selected_df.to_csv('direct_supply_data.csv', index=False)

token: FeqFZskZp2uk219UKZhPazt7cKjpSJDAcRPQ73xQ
status_code: 200
615
token: QUaZ7R6hkFSMLXucwHFiUS6OFTjpoPXgBVcvCizG
status_code: 200
539
token: Vdvnfaw809UubMW4DR3jBEqSEQVKyGe2fYq1z1IW
status_code: 200
144
token: GiUv85Nqxee9DUbQeMs3zb8QJpcE4EMDT8fpyFOE
status_code: 200
32
token: TSGrhdaf3QzdLBxrrtFpoPA9plMf9YLvFBnTo0aQ
status_code: 200
8
